In [1]:
from pathlib import Path

import torch

from dynamic_fusion.network_trainer.configuration import TrainerConfiguration
from dynamic_fusion.network_trainer.network_loader import NetworkLoader

from pathlib import Path
from dynamic_fusion.utils.dataset import CocoTestDataset
from dynamic_fusion.utils.dataset import collate_test_items
from dynamic_fusion.utils.evaluation import get_metrics


dataset_path = Path('..', 'data', 'interim', 'coco', 'test', '2subbins')


metrics_temporal_scale = []
spatial_scale = 1
temporal_scale = 1
for spatial_scale in range(1,4):
    dataset = CocoTestDataset(dataset_path, (spatial_scale, spatial_scale), threshold=1.3)

/home/davton/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
events = []

with h5py.File(self.directory_list[index] / f"events.h5", "r") as file:
    discretized_events = DiscretizedEvents.load_from_file(file)


In [5]:
bin_size = 0.002

count = dataset[0][7]
count.shape

torch.Size([150, 2, 106, 106])

In [8]:
5/100


0.05